In [1]:
import numpy as np
import pandas as pd
from MnistDataloader import MnistDataloader
from oneNNClassifier import oneNNClassifier
from utilities import random_sample
from os.path  import join
import timeit
from sklearn.metrics import accuracy_score
from sklearn.metrics.pairwise import cosine_similarity
import json
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [2]:
input_path = './dataset/'
training_images_filepath = join(input_path, 'train-images-idx3-ubyte')
training_labels_filepath = join(input_path, 'train-labels-idx1-ubyte')
test_images_filepath = join(input_path, 't10k-images-idx3-ubyte')
test_labels_filepath = join(input_path, 't10k-labels-idx1-ubyte')

In [3]:
mnist_dataloader = MnistDataloader(training_images_filepath, training_labels_filepath, test_images_filepath, test_labels_filepath)
(x_train, y_train), (x_test, y_test) = mnist_dataloader.load_data()
x_train = np.array([np.hstack(x).astype(np.float32) for x in x_train])
x_test = np.array([np.hstack(x).astype(np.float32) for x in x_test])
y_train = np.array(y_train, np.float32)
y_test = np.array(y_test, np.float32)

In [4]:
type(x_train[0][1]), type(y_train[0])

(numpy.float32, numpy.float32)

In [5]:
len(x_train), len(y_train), len(x_test), len(y_test), len(x_train[0]), y_train[0]

(60000, 60000, 10000, 10000, 784, 5.0)

In [6]:
def perform_pca(X, n_components=50):
    pca = PCA(n_components=n_components, random_state=42)
    X_reduced = pca.fit_transform(X)
    return X_reduced, pca

def facility_location_selection(X_reduced, k):
    n_samples = X_reduced.shape[0]
    selected_indices = []

    # Distances to the nearest selected prototype for each sample
    # Initialize to inf since we haven't chosen any prototype yet
    min_distances = np.full(n_samples, np.inf)

    for _ in range(k):
        best_candidate = None
        best_improvement = 0.0

        # Current coverage cost is the sum of min distances
        current_coverage = np.sum(min_distances)

        for i in range(n_samples):
            # Skip points already selected
            if i in selected_indices:
                continue

            # Compute distances from X[i] to all samples
            d = np.linalg.norm(X_reduced - X_reduced[i], axis=1)

            # If we were to add i, the new distance for each sample j would be:
            #   min( existing distance, distance to X[i] )
            new_distances = np.minimum(min_distances, d)

            # New coverage cost (sum of these min distances)
            new_coverage = np.sum(new_distances)

            # Improvement = how much we reduce total distance
            improvement = current_coverage - new_coverage
            if improvement > best_improvement:
                best_improvement = improvement
                best_candidate = i

        # After checking all samples, pick the point that gives the max improvement
        selected_indices.append(best_candidate)

        # Update min_distances array with distances to the newly added prototype
        d_new_center = np.linalg.norm(X_reduced - X_reduced[best_candidate], axis=1)
        min_distances = np.minimum(min_distances, d_new_center)

    return selected_indices

def select_k_facilities_per_class(X, y, k_per_class=10, n_components=50):

    # Reduce dimensionality of the entire dataset
    X_reduced, pca_model = perform_pca(X, n_components=n_components)
    
    unique_classes = np.unique(y)
    selected_indices = []

    for c in unique_classes:
        # Extract the indices for class c
        class_indices = np.where(y == c)[0]
        X_c_reduced = X_reduced[class_indices]

        # Run facility location selection in the reduced space
        k_facility_indices = facility_location_selection(X_c_reduced, k_per_class)

        # Map back to overall dataset indices
        selected_indices_c = class_indices[k_facility_indices]
        selected_indices.extend(selected_indices_c)
    
    return selected_indices

In [8]:
# train set is sampled using M/10 prototypes per class

# sample_sizes = [10, 20, 30, 40, 50]
reduced_dims = [3, 4, 8, 16, 64, 128, 256, 512, 784]
sample_sizes = [100, 500, 1000, 2000, 5000, 10000]
storage = {} 
execution_data = {d:[] for d in reduced_dims}

for dim in reduced_dims:
    print('Reduction Dimension:', dim)
    for M in sample_sizes:
        # Sample prototypes
        selected_idxs = select_k_facilities_per_class(x_train, y_train, k_per_class= int(M/10), n_components=dim)
        x_sample, y_sample = x_train[selected_idxs], y_train[selected_idxs]

        # Model
        model = oneNNClassifier(x_sample, y_sample)
        
        # Timing
        elapsed_time = timeit.timeit(lambda: model.predict(x_test, size=M, storage=storage, weighted=False), 
                                number=1)
        # Accuracy
        accuracy = accuracy_score(y_test, storage[M])

        print(f"\tSample size: {M}, Accuracy: {accuracy:.2f}, Execution time: {elapsed_time:.4f} seconds")
        # Store Data
        execution_data[dim].append({"sample_size": M, "time": elapsed_time, "accuracy": accuracy})

Reduction Dimension: 3
	Sample size: 100, Accuracy: 0.76, Execution time: 1.8268 seconds
	Sample size: 500, Accuracy: 0.86, Execution time: 8.4982 seconds
	Sample size: 1000, Accuracy: 0.89, Execution time: 14.3457 seconds
	Sample size: 2000, Accuracy: 0.91, Execution time: 28.1585 seconds
	Sample size: 5000, Accuracy: 0.94, Execution time: 85.9831 seconds
	Sample size: 10000, Accuracy: 0.95, Execution time: 141.7501 seconds
Reduction Dimension: 4
	Sample size: 100, Accuracy: 0.77, Execution time: 1.4581 seconds
	Sample size: 500, Accuracy: 0.87, Execution time: 7.1050 seconds
	Sample size: 1000, Accuracy: 0.89, Execution time: 14.3195 seconds
	Sample size: 2000, Accuracy: 0.92, Execution time: 28.4070 seconds
	Sample size: 5000, Accuracy: 0.94, Execution time: 70.8274 seconds
	Sample size: 10000, Accuracy: 0.95, Execution time: 142.5174 seconds
Reduction Dimension: 8
	Sample size: 100, Accuracy: 0.81, Execution time: 1.4573 seconds
	Sample size: 500, Accuracy: 0.89, Execution time: 7.

KeyboardInterrupt: 

In [9]:
with open("execution_data_facility_location.json", "w") as file:
    json.dump(execution_data, file, indent=4)